# Drowsiness Detection Method Overview

This project implements a drowsiness detection system that relies on monitoring the **Eye Aspect Ratio (EAR)**—a simple yet effective metric to determine whether a person’s eyes are open or closed.

### Key Concepts

- **Facial Landmark Detection:**
  The system utilizes a pre-trained facial landmark predictor to locate key points on a person’s face. These points, especially those outlining the eyes, are critical for subsequent analysis.

- **Calculating the Eye Aspect Ratio (EAR):**
  The EAR is computed by comparing the vertical distances (between the eyelid landmarks) to the horizontal distance (between the corners of the eyes). Essentially, when the eyes are open, the vertical distances are relatively large compared to the horizontal distance. When the eyes begin to close, these vertical distances decrease, causing the EAR to drop.

- **Determining Drowsiness:**
  By monitoring the EAR over consecutive frames in a video stream, the system can detect when the eyes remain closed for an extended period. A consistently low EAR signals that the individual might be drowsy, at which point an alarm is activated to prompt alertness.

This approach, based on real-time computer vision techniques, offers a reliable and efficient way to monitor and address drowsiness, especially in applications like driver alertness systems.

![](https://learnopencv.com/wp-content/uploads/2022/09/03-driver-drowsiness-detection-EAR-points.png)


In [7]:
# Import necessary libraries
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import pygame
import time

In [8]:
# Function to calculate the Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)


In [25]:
# Constants
MODEL_PATH = "shape_predictor_68_face_landmarks.dat"
ALARM_PATH = "assets_alarm.mp3"
THRESH = 0.25
FLAG_THRESH = 10

In [26]:
# Initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(MODEL_PATH)

pygame.mixer.init()
pygame.mixer.music.load(ALARM_PATH)

In [27]:
# Get eye landmark indexes
print(imutils.face_utils.FACIAL_LANDMARKS_68_IDXS)
(rStart, rEnd)=imutils.face_utils.FACIAL_LANDMARKS_68_IDXS['right_eye']
(lStart, lEnd)=imutils.face_utils.FACIAL_LANDMARKS_68_IDXS['left_eye']

OrderedDict([('mouth', (48, 68)), ('inner_mouth', (60, 68)), ('right_eyebrow', (17, 22)), ('left_eyebrow', (22, 27)), ('right_eye', (36, 42)), ('left_eye', (42, 48)), ('nose', (27, 36)), ('jaw', (0, 17))])


In [46]:
# Start video capture
cap = cv2.VideoCapture(0)
flag = 0
ALARM_ON = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face) #its type is object detection object, so to get the index for the eyes we need it to be in array format
        landmarks = face_utils.shape_to_np(landmarks)

        rightEye = landmarks[rStart:rEnd]
        leftEye = landmarks[lStart:lEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        # Draw convex hull around each eye for visual feedback
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if ear < THRESH:
            flag += 1
            print(flag)
            if flag >= FLAG_THRESH :
                time.sleep(.01)
                cv2.putText(frame, "****************Drowsy!****************", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, 'Sleeping Time: 00:' + str(flag) + ' sec', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                pygame.mixer.music.play()
        else:
            flag = 0

    cv2.imshow("Frame", frame)

    if cv2.waitKey(30) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1
1
1
1
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
1
2
3
4
5
6
7
8
9
10
11
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
